In [1]:
from PipelineTS.dataset import LoadWebSales

In [2]:
init_data = LoadWebSales()[['date', 'type_a']]

In [3]:
valid_data = init_data.iloc[-61:, :]
data = init_data.iloc[:-61, :]
data.tail(5)

,date,type_a
2023,2021-07-10,10094
2024,2021-07-11,9603
2025,2021-07-12,10700
2026,2021-07-13,9854
2027,2021-07-14,9486


# WideGBRTModel

In [4]:
from PipelineTS.ml_model import WideGBRTModel

wide_gbrt = WideGBRTModel(time_col='date', target_col='type_a', lags=30, random_state=42, quantile=0.9)
wide_gbrt.fit(data)

In [5]:
wide_gbrt.predict(31)

,date,type_a,type_a_lower,type_a_upper
0,2021-07-15,8749.498207,7582.506411,9916.490004
1,2021-07-16,10538.456234,9132.856543,11944.055925
2,2021-07-17,10927.639383,9470.131168,12385.147599
3,2021-07-18,11712.159335,10150.013308,13274.305362
4,2021-07-19,12658.266367,10969.930344,14346.602390
5,2021-07-20,12436.389973,10777.647410,14095.132536
6,2021-07-21,13127.083656,11376.217655,14877.949658
7,2021-07-22,12045.810946,10439.163087,13652.458804
8,2021-07-23,11432.476142,9907.633739,12957.318545
9,2021-07-24,10907.364984,9452.560930,12362.169037


# ProphetModel

In [6]:
from PipelineTS.statistic_model import ProphetModel
model = ProphetModel(time_col='date', target_col='type_a', quantile=0.9)

In [7]:
model.fit(data)

In [8]:
model.predict(30)

,date,type_a,type_a_lower,type_a_upper
0,2021-07-15,7799.317820,7699.014535,7899.621104
1,2021-07-16,7276.720797,7183.138382,7370.303211
2,2021-07-17,5125.854089,5059.932939,5191.775239
3,2021-07-18,4876.969166,4814.248805,4939.689526
4,2021-07-19,9425.121478,9303.909513,9546.333443
5,2021-07-20,8911.714179,8797.104899,9026.323459
6,2021-07-21,8511.672969,8402.208427,8621.137511
7,2021-07-22,8180.260887,8075.058477,8285.463297
8,2021-07-23,7737.343102,7637.836845,7836.849359
9,2021-07-24,5633.627255,5561.175878,5706.078632


# AutoARIMA

In [9]:
from PipelineTS.statistic_model import AutoARIMAModel

In [10]:
auto_arima = AutoARIMAModel(time_col='date', target_col='type_a', quantile=0.9)
auto_arima.fit(data=data)

In [11]:
auto_arima.predict(40)

,date,type_a,type_a_lower,type_a_upper
0,2021-07-15,9613.791349,7628.260761,11599.321938
1,2021-07-16,9607.376926,7623.171105,11591.582747
2,2021-07-17,10047.700287,7972.554745,12122.845829
3,2021-07-18,9904.772544,7859.145784,11950.399305
4,2021-07-19,10255.825701,8137.696141,12373.955260
5,2021-07-20,10007.131167,7940.364341,12073.897993
6,2021-07-21,9863.232608,7826.185066,11900.280151
7,2021-07-22,9939.151044,7886.424113,11991.877976
8,2021-07-23,9959.854247,7902.851496,12016.856999
9,2021-07-24,10077.331445,7996.066198,12158.596693


# GBDT

In [12]:
from PipelineTS.ml_model import *

## CatBoost

In [13]:
cbt = CatBoostModel(time_col='date', target_col='type_a', lags=30, random_state=42, quantile=0.9)
cbt.fit(data=data)

In [14]:
cbt.predict(30)

,date,type_a,type_a_lower,type_a_upper
0,2021-07-15,10116.587161,9568.361423,11156.102723
1,2021-07-16,11020.208037,10517.793382,11790.746830
2,2021-07-17,10810.933551,10366.224177,12430.527327
3,2021-07-18,11096.973427,10726.696870,12141.420453
4,2021-07-19,11075.392439,10539.818956,13067.411089
5,2021-07-20,11120.897243,10554.098426,12854.736258
6,2021-07-21,11681.372311,11064.011450,13200.723904
7,2021-07-22,11919.195378,11182.472450,13312.469791
8,2021-07-23,12079.207460,11279.408661,14465.130339
9,2021-07-24,12794.387701,11660.091064,14618.836283


## XGBoost

In [15]:
xgb = XGBoostModel(time_col='date', target_col='type_a', lags=30, random_state=42, quantile=0.9)
xgb.fit(data=data)

In [16]:
xgb.predict(30)

,date,type_a,type_a_lower,type_a_upper
0,2021-07-15,10654.655273,8261.059570,13048.250977
1,2021-07-16,12009.849609,9311.806641,14707.892578
2,2021-07-17,11506.630859,8921.636719,14091.625000
3,2021-07-18,11093.751953,8601.512695,13585.991211
4,2021-07-19,11819.616211,9164.309570,14474.922852
5,2021-07-20,13092.085938,10150.916016,16033.255859
6,2021-07-21,14712.363281,11407.193359,18017.533203
7,2021-07-22,13415.467773,10401.649414,16429.287109
8,2021-07-23,13607.886719,10550.840820,16664.933594
9,2021-07-24,13951.577148,10817.320312,17085.833984


## LightGBM

In [17]:
lgb = LightGBMModel(time_col='date', target_col='type_a', lags=30, random_state=42, quantile=0.9)
lgb.fit(data=data)

In [18]:
lgb.predict(30)

,date,type_a,type_a_lower,type_a_upper
0,2021-07-15,10154.671712,9109.796994,11638.569512
1,2021-07-16,10289.449992,8771.675034,12276.503823
2,2021-07-17,10319.511157,8751.073256,12604.526509
3,2021-07-18,10348.146822,9247.544093,13757.795071
4,2021-07-19,11698.661485,9704.824230,13213.340909
5,2021-07-20,11349.030544,10441.121268,13600.255447
6,2021-07-21,11538.765056,11501.437149,14372.343618
7,2021-07-22,11900.846536,11672.142466,14676.950801
8,2021-07-23,12377.892438,11768.474257,14736.702548
9,2021-07-24,12766.077846,12420.983293,15500.235561


## RandomForest

In [19]:
rf = RandomForestModel(time_col='date', target_col='type_a', lags=30, random_state=42, quantile=0.9)
rf.fit(data=data)

In [20]:
rf.predict(30)

,date,type_a,type_a_lower,type_a_upper
0,2021-07-15,11214.85,8850.845398,13578.854602
1,2021-07-16,11564.08,9126.460385,14001.699615
2,2021-07-17,12039.05,9501.310342,14576.789658
3,2021-07-18,12642.92,9977.889164,15307.950836
4,2021-07-19,12863.96,10152.335623,15575.584377
5,2021-07-20,12780.77,10086.681439,15474.858561
6,2021-07-21,14210.24,11214.830096,17205.649904
7,2021-07-22,14214.28,11218.018495,17210.541505
8,2021-07-23,13713.79,10823.027959,16604.552041
9,2021-07-24,13954.97,11013.369060,16896.570940


# NN Model

In [4]:
from PipelineTS.nn_model import *

## DLinearModel

In [22]:
d_linear = DLinearModel(time_col='date', target_col='type_a', lags=30, random_state=42, quantile=0.9)

In [23]:
d_linear.fit(data)
d_linear.predict(30)

,date,type_a,type_a_lower,type_a_upper
0,2021-07-15,9544.323242,7684.914062,11403.732422
1,2021-07-16,10001.357422,8052.909668,11949.805664
2,2021-07-17,9959.427734,8019.148926,11899.707031
3,2021-07-18,9585.663086,7718.200195,11453.125977
4,2021-07-19,9950.253906,8011.762207,11888.746094
5,2021-07-20,9850.069336,7931.095215,11769.043945
6,2021-07-21,9985.506836,8040.146973,11930.867188
7,2021-07-22,9700.318359,7810.518555,11590.118164
8,2021-07-23,9890.510742,7963.658203,11817.364258
9,2021-07-24,9858.060547,7937.529785,11778.591797


##  NLinearModel

In [24]:
n_linear = NLinearModel(time_col='date', target_col='type_a', lags=30, random_state=42, quantile=0.9)

In [25]:
n_linear.fit(data)
n_linear.predict(30)

,date,type_a,type_a_lower,type_a_upper
0,2021-07-15,9820.378906,7881.794434,11758.963867
1,2021-07-16,9875.342773,7925.908203,11824.777344
2,2021-07-17,10041.634766,8059.373047,12023.896484
3,2021-07-18,9965.751953,7998.470215,11933.034180
4,2021-07-19,10491.192383,8420.186523,12562.198242
5,2021-07-20,10176.806641,8167.861816,12185.751953
6,2021-07-21,10082.493164,8092.166016,12072.820312
7,2021-07-22,9989.190430,8017.281738,11961.099609
8,2021-07-23,10057.761719,8072.316895,12043.207031
9,2021-07-24,9888.271484,7936.284668,11840.258789


##  NBeatsModel 

In [26]:
n_beats = NBeatsModel(time_col='date', target_col='type_a', lags=30, random_state=42, quantile=0.9)

In [27]:
n_beats.fit(data)
n_beats.predict(30)

,date,type_a,type_a_lower,type_a_upper
0,2021-07-15,10272.398438,7965.511230,12579.285156
1,2021-07-16,11184.761719,8672.984375,13696.539062
2,2021-07-17,11067.205078,8581.827148,13552.582031
3,2021-07-18,11671.572266,9050.470703,14292.672852
4,2021-07-19,11611.823242,9004.139648,14219.505859
5,2021-07-20,10955.768555,8495.416016,13416.120117
6,2021-07-21,10373.398438,8043.829590,12702.966797
7,2021-07-22,10732.993164,8322.669922,13143.315430
8,2021-07-23,10322.168945,8004.104980,12640.232422
9,2021-07-24,10557.044922,8186.234375,12927.854492


## TCNModel

In [28]:
tcn = TCNModel(time_col='date', target_col='type_a', lags=30, random_state=42, quantile=0.9)

In [29]:
tcn.fit(data)
tcn.predict(30)

,date,type_a,type_a_lower,type_a_upper
0,2021-07-15,11688.364258,7920.665039,15456.062500
1,2021-07-16,10687.386719,7242.348633,14132.423828
2,2021-07-17,11546.291016,7824.388672,15268.192383
3,2021-07-18,11782.465820,7984.433105,15580.498047
4,2021-07-19,11610.870117,7868.150879,15353.588867
5,2021-07-20,12264.043945,8310.776367,16217.310547
6,2021-07-21,11419.899414,7738.738770,15101.059570
7,2021-07-22,10598.820312,7182.331543,14015.308594
8,2021-07-23,11025.179688,7471.255371,14579.103516
9,2021-07-24,10631.201172,7204.274414,14058.126953


## NHitsModel

In [30]:
n_hits = NHitsModel(time_col='date', target_col='type_a', lags=30, random_state=42, quantile=0.9)

In [31]:
n_hits.fit(data)
n_hits.predict(30)

,date,type_a,type_a_lower,type_a_upper
0,2021-07-15,11207.983398,8469.210938,13946.756836
1,2021-07-16,11426.943359,8634.666016,14219.221680
2,2021-07-17,10941.923828,8268.165039,13615.682617
3,2021-07-18,10805.572266,8165.132812,13446.012695
4,2021-07-19,11093.601562,8382.779297,13804.424805
5,2021-07-20,10775.504883,8142.412598,13408.597656
6,2021-07-21,10525.598633,7953.573242,13097.625000
7,2021-07-22,10371.640625,7837.236328,12906.045898
8,2021-07-23,10273.091797,7762.768555,12783.416016
9,2021-07-24,10500.330078,7934.479004,13066.181641


## TFTModel

In [32]:
tft = TFTModel(time_col='date', target_col='type_a', lags=30, random_state=42, quantile=0.9)

In [33]:
tft.fit(data)
tft.predict(30)

,date,type_a,type_a_lower,type_a_upper
0,2021-07-15,2152.076904,-18890.138672,23194.292969
1,2021-07-16,2152.076660,-18890.136719,23194.289062
2,2021-07-17,2152.076660,-18890.136719,23194.289062
3,2021-07-18,2152.076660,-18890.136719,23194.289062
4,2021-07-19,2152.076660,-18890.136719,23194.289062
5,2021-07-20,2152.076904,-18890.138672,23194.292969
6,2021-07-21,2152.076904,-18890.138672,23194.292969
7,2021-07-22,2152.076660,-18890.136719,23194.289062
8,2021-07-23,2152.076660,-18890.136719,23194.289062
9,2021-07-24,2152.076660,-18890.136719,23194.289062


## GAU

In [4]:
from PipelineTS.nn_model import *

In [5]:
gau_model = GAUModel(time_col='date', target_col='type_a', lags=30, random_state=42, 
                     quantile=0.9, device='cpu', patience=100, verbose=True)
gau_model.fit(data, valid_data)

Using cpu backend
Epoch 1/1000  
62/62 - p0 - loss: 2279.6980 - mae: 8824.5469 - val_loss: 997.0201 - val_mae: 997.0201 - 0.22s/epoch - 0.004s/step
Epoch 2/1000  
62/62 - p0 - loss: 2679.0811 - mae: 7564.3237 - val_loss: 894.9512 - val_mae: 894.9512 - 0.19s/epoch - 0.003s/step
Epoch 3/1000  
62/62 - p0 - loss: 2440.7502 - mae: 7177.9313 - val_loss: 833.5668 - val_mae: 833.5668 - 0.19s/epoch - 0.003s/step
Epoch 4/1000  
62/62 - p1 - loss: 2529.2200 - mae: 7179.4312 - val_loss: 843.4922 - val_mae: 843.4922 - 0.18s/epoch - 0.003s/step
Epoch 5/1000  
62/62 - p2 - loss: 2546.4729 - mae: 7120.2011 - val_loss: 909.0696 - val_mae: 909.0696 - 0.19s/epoch - 0.003s/step
Epoch 6/1000  
62/62 - p3 - loss: 2554.9485 - mae: 7019.3629 - val_loss: 849.7239 - val_mae: 849.7238 - 0.19s/epoch - 0.003s/step
Epoch 7/1000  
62/62 - p0 - loss: 2504.5815 - mae: 7009.7169 - val_loss: 816.3682 - val_mae: 816.3684 - 0.19s/epoch - 0.003s/step
Epoch 8/1000  
62/62 - p0 - loss: 2543.0977 - mae: 6954.6010 - val_loss:

In [6]:
gau_model.predict(31)

,date,type_a,type_a_lower,type_a_upper
0,2021-07-15,9913.943359,8118.337531,11709.549188
1,2021-07-16,9976.311523,8169.409620,11783.213426
2,2021-07-17,9186.514648,7522.660151,10850.369146
3,2021-07-18,8298.921875,6795.827501,9802.016249
4,2021-07-19,9083.984375,7438.700082,10729.268668
5,2021-07-20,8420.651367,6895.509440,9945.793294
6,2021-07-21,8687.975586,7114.416101,10261.535071
7,2021-07-22,8338.649414,6828.359618,9848.939210
8,2021-07-23,8358.902344,6844.944353,9872.860335
9,2021-07-24,9515.394531,7791.973561,11238.815502


## StackingRNN

In [7]:
rnn_model = StackingRNNModel(time_col='date', target_col='type_a', lags=30, random_state=42, 
                     quantile=0.9, device='cpu', patience=10, verbose=True, use_standard_scaler=True)
rnn_model.fit(data, valid_data)

Using cpu backend
Epoch 1/1000  
62/62 - p0 - loss: 10211.0537 - mae: 27050.8503 - val_loss: 7974.4155 - val_mae: 7974.4160 - 0.52s/epoch - 0.008s/step
Epoch 2/1000  
62/62 - p0 - loss: 8596.8486 - mae: 26101.1911 - val_loss: 6337.0879 - val_mae: 6337.0869 - 0.50s/epoch - 0.008s/step
Epoch 3/1000  
62/62 - p0 - loss: 5278.8999 - mae: 23673.1009 - val_loss: 2986.6169 - val_mae: 2986.6172 - 0.51s/epoch - 0.008s/step
Epoch 4/1000  
62/62 - p0 - loss: 1550.8651 - mae: 19443.2578 - val_loss: 2420.5244 - val_mae: 2420.5244 - 0.51s/epoch - 0.008s/step
Epoch 5/1000  
62/62 - p1 - loss: 6379.6509 - mae: 14799.7443 - val_loss: 8515.8027 - val_mae: 8515.8027 - 0.50s/epoch - 0.008s/step
Epoch 6/1000  
62/62 - p2 - loss: 11522.0908 - mae: 11850.9023 - val_loss: 13639.9287 - val_mae: 13639.9277 - 0.50s/epoch - 0.008s/step
Epoch 7/1000  
62/62 - p3 - loss: 15776.5859 - mae: 10047.8209 - val_loss: 17854.3574 - val_mae: 17854.3613 - 0.50s/epoch - 0.008s/step
Epoch 8/1000  
62/62 - p4 - loss: 16858.5527

In [8]:
rnn_model.predict(31)

,date,type_a,type_a_lower,type_a_upper
0,2021-07-15,10784.322266,791.308714,20777.335817
1,2021-07-16,10740.674805,788.106045,20693.243565
2,2021-07-17,10746.267578,788.516419,20704.018737
3,2021-07-18,10787.979492,791.577066,20784.381918
4,2021-07-19,10748.519531,788.681658,20708.357404
5,2021-07-20,10755.156250,789.168633,20721.143867
6,2021-07-21,10781.309570,791.087655,20771.531486
7,2021-07-22,10766.059570,789.968673,20742.150467
8,2021-07-23,10778.210938,790.860290,20765.561585
9,2021-07-24,10742.849609,788.265623,20697.433596


## Time2Vec

In [9]:
time2vec_model = Time2VecModel(time_col='date', target_col='type_a', lags=30, random_state=42, 
                     quantile=0.9, device='cpu', patience=10, verbose=True)
time2vec_model.fit(data, valid_data)

Using cpu backend
Epoch 1/1000  
62/62 - p0 - loss: 2070.5344 - mae: 8664.4192 - val_loss: 1232.7745 - val_mae: 1232.7747 - 0.24s/epoch - 0.004s/step
Epoch 2/1000  
62/62 - p0 - loss: 2744.4741 - mae: 7622.5519 - val_loss: 934.9744 - val_mae: 934.9744 - 0.24s/epoch - 0.004s/step
Epoch 3/1000  
62/62 - p0 - loss: 2778.8848 - mae: 7201.0429 - val_loss: 580.3691 - val_mae: 580.3691 - 0.24s/epoch - 0.004s/step
Epoch 4/1000  
62/62 - p0 - loss: 2701.6162 - mae: 6563.3655 - val_loss: 479.7181 - val_mae: 479.7181 - 0.24s/epoch - 0.004s/step
Epoch 5/1000  
62/62 - p0 - loss: 2486.6365 - mae: 6491.6393 - val_loss: 473.1888 - val_mae: 473.1888 - 0.24s/epoch - 0.004s/step
Epoch 6/1000  
62/62 - p1 - loss: 2437.4285 - mae: 6307.7085 - val_loss: 505.1319 - val_mae: 505.1319 - 0.24s/epoch - 0.004s/step
Epoch 7/1000  
62/62 - p2 - loss: 2454.4146 - mae: 6312.5639 - val_loss: 522.7848 - val_mae: 522.7848 - 0.24s/epoch - 0.004s/step
Epoch 8/1000  
62/62 - p3 - loss: 2492.0991 - mae: 6083.8634 - val_los

In [10]:
time2vec_model.predict(31)

,date,type_a,type_a_lower,type_a_upper
0,2021-07-15,9193.344727,7843.466473,10543.222980
1,2021-07-16,8872.250977,7569.519597,10174.982356
2,2021-07-17,8951.838867,7637.421430,10266.256305
3,2021-07-18,9181.779297,7833.599220,10529.959374
4,2021-07-19,9575.614258,8169.606560,10981.621956
5,2021-07-20,9677.299805,8256.361403,11098.238206
6,2021-07-21,9743.314453,8312.682981,11173.945925
7,2021-07-22,9876.368164,8426.200134,11326.536194
8,2021-07-23,10199.241211,8701.665048,11696.817373
9,2021-07-24,10086.958008,8605.868626,11568.047390


## TransformerModel

In [5]:
transformer = TransformerModel(time_col='date', target_col='type_a', lags=30, random_state=42, 
                               quantile=0.9, enable_progress_bar=True, enable_model_summary=True)

In [ ]:
transformer.fit(data)


  | Name                | Type                | Params
------------------------------------------------------------
0 | criterion           | MSELoss             | 0     
1 | train_metrics       | MetricCollection    | 0     
2 | val_metrics         | MetricCollection    | 0     
3 | encoder             | Linear              | 128   
4 | positional_encoding | _PositionalEncoding | 0     
5 | transformer         | Transformer         | 548 K 
6 | decoder             | Linear              | 1.9 K 
------------------------------------------------------------
550 K     Trainable params
0         Non-trainable params
550 K     Total params
2.203     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]


  | Name                | Type                | Params
------------------------------------------------------------
0 | criterion           | MSELoss             | 0     
1 | train_metrics       | MetricCollection    | 0     
2 | val_metrics         | MetricCollection    | 0     
3 | encoder             | Linear              | 128   
4 | positional_encoding | _PositionalEncoding | 0     
5 | transformer         | Transformer         | 548 K 
6 | decoder             | Linear              | 1.9 K 
------------------------------------------------------------
550 K     Trainable params
0         Non-trainable params
550 K     Total params
2.203     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]


  | Name                | Type                | Params
------------------------------------------------------------
0 | criterion           | MSELoss             | 0     
1 | train_metrics       | MetricCollection    | 0     
2 | val_metrics         | MetricCollection    | 0     
3 | encoder             | Linear              | 128   
4 | positional_encoding | _PositionalEncoding | 0     
5 | transformer         | Transformer         | 548 K 
6 | decoder             | Linear              | 1.9 K 
------------------------------------------------------------
550 K     Trainable params
0         Non-trainable params
550 K     Total params
2.203     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]


  | Name                | Type                | Params
------------------------------------------------------------
0 | criterion           | MSELoss             | 0     
1 | train_metrics       | MetricCollection    | 0     
2 | val_metrics         | MetricCollection    | 0     
3 | encoder             | Linear              | 128   
4 | positional_encoding | _PositionalEncoding | 0     
5 | transformer         | Transformer         | 548 K 
6 | decoder             | Linear              | 1.9 K 
------------------------------------------------------------
550 K     Trainable params
0         Non-trainable params
550 K     Total params
2.203     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

In [ ]:
transformer.predict(31)

# MultiOutputRegressorModel

In [6]:
from PipelineTS.ml_model import MultiOutputRegressorModel

In [7]:
from xgboost import XGBRegressor

mor_model = MultiOutputRegressorModel(time_col='date', target_col='type_a', lags=30, random_state=42, 
                     quantile=0.9, estimator=XGBRegressor)
mor_model.fit(data)

In [8]:
mor_model.predict(31)

,date,type_a,type_a_lower,type_a_upper
0,2021-07-15,10239.683594,8479.528481,11999.838707
1,2021-07-16,12448.630859,10308.767742,14588.493977
2,2021-07-17,11585.619141,9594.103819,13577.134462
3,2021-07-18,12246.194336,10141.129138,14351.259534
4,2021-07-19,12731.150391,10542.723449,14919.577332
5,2021-07-20,12518.515625,10366.639634,14670.391616
6,2021-07-21,13183.864258,10917.617858,15450.110658
7,2021-07-22,12402.720703,10270.749333,14534.692073
8,2021-07-23,11575.401367,9585.642434,13565.160300
9,2021-07-24,11677.517578,9670.205332,13684.829824


# MultiStepRegressorModel

In [4]:
from PipelineTS.ml_model.multi_output_model import MultiStepRegressorModel
from xgboost import XGBRegressor

msr_model = MultiStepRegressorModel(time_col='date', target_col='type_a', lags=30, random_state=42, 
                     quantile=0.9, estimator=XGBRegressor)
msr_model.fit(data)

In [5]:
msr_model.predict(31)

,date,type_a,type_a_lower,type_a_upper
0,2021-07-15,10239.683594,8524.521390,11954.845797
1,2021-07-16,11958.166992,9955.156268,13961.177716
2,2021-07-17,11706.451172,9745.603221,13667.299122
3,2021-07-18,11311.289062,9416.631352,13205.946773
4,2021-07-19,11581.852539,9641.875045,13521.830034
5,2021-07-20,11520.117188,9590.480456,13449.753919
6,2021-07-21,11327.683594,9430.279775,13225.087413
7,2021-07-22,11566.968750,9629.484312,13504.453188
8,2021-07-23,10466.619141,8713.444896,12219.793385
9,2021-07-24,9852.848633,8202.481860,11503.215406
